In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from pyprojroot import here
from pathlib import Path


In [ ]:
from gitdata.dataset import Dataset

ds = Dataset(root_dir=here() / "data" / "dummy", dataset_name="test_dataset")
ds.current_commit
ds.checkout()

In [ ]:

ds.commit(commit_message="test from empty commit", add_files=[here() / "README.md", here() / ".pre-commit-config.yaml"])
print(ds.file_dict, ds.current_commit.version_hash)


In [ ]:

ds.commit(commit_message="test removing a file", remove_files="README.md")
print("-----")
print(ds.file_dict, ds.current_commit.version_hash)


In [ ]:

ds.commit(commit_message="test adding a config file", add_files=here() / ".flake8")
print("-----")
print(ds.file_dict, ds.current_commit.version_hash)


In [ ]:

ds.checkout()
print("-----")
print(ds.file_dict, ds.current_commit.version_hash)


In [ ]:
# Prototype finding latest commit hash
# We do this by identifying the commit (based on commit.json) that is not a parent to any other commit.
import json5 as json


def latest_version_hash(self):
    jsons = list(self.dataset_dir.glob("*/commit.json"))
    commit_to_parent = dict()
    for json_file in jsons:
        with open(json_file, "r") as f:
            data = json.loads(f.read())
            commit_to_parent[data["version_hash"]] = data["parent_hash"]

    # Now, find the commit that is not a parent to any other commit.
    for k, v in commit_to_parent.items():
        if v not in commit_to_parent.values():
            break
    return k

In [ ]:
ds.checkout()
ds.file_dict, ds.current_commit.version_hash # need to assert that it is not empty

In [ ]:
from gitdata.catalog import Catalog

c = Catalog(root_dir=here() / "data" / "dummy")
c.datasets()

ds = c.get_dataset("test_dataset")
ds.checkout()

ds.file_dict


In [ ]:
ds2 = c.create_dataset("another dataset", description="a test dataset")
ds2.commit(commit_message="testing another dataset", add_files=[here() / "README.md", here() / ".pre-commit-config.yaml"])
ds2.file_dict

In [ ]:
ds2.current_version_hash()

In [ ]:
oops